In [ ]:
# Task 1: Data Cleaning and Preprocessing
# Objective: Clean and prepare a raw dataset (with nulls, duplicates, inconsistent formats).
# Tools: Excel / Python (Pandas)
# Deliverables: Cleaned dataset + short summary of changes
# Hints / Mini Guide:
# Identify and handle missing values using .isnull() in Python or filters in Excel.
# Remove duplicate rows using .drop_duplicates() or Excel’s “Remove Duplicates”.
# Standardize text values like gender, country names, etc.
# Convert date formats to a consistent type (e.g., dd-mm-yyyy).
# Rename column headers to be clean and uniform (e.g., lowercase, no spaces).
# Check and fix data types (e.g., age should be int, date as datetime).
# Dataset names from Kaggle suitable for Task 1:
# Customer Personality Analysis
# Medical Appointment No Shows
# Mall Customer Segmentation Data
# Netflix Movies and TV Shows
# Sales Data

In [46]:
#importing necessary libraries after installing them
%pip install pandas numpy matplotlib seaborn openpyxl
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

You should consider upgrading via the 'c:\Users\sudhakar\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [47]:
# function to extract zipfile
from zipfile import ZipFile
import os

def extract_zip(zip_path,extract_to):
    with ZipFile(zip_path,"r") as zip:
        zip.extractall(extract_to)
        print("extraction complete")
    return zip.namelist()

# making output directory to extract the zip file
extract_to="./AmazonSalesData"
os.makedirs(extract_to, exist_ok=True)

extract_zip("amazon.csv.zip",extract_to)

extraction complete


['amazon.csv']

In [48]:
#Function to read the csv file
def read_csv(file_path):
    try:
        df=pd.read_csv(file_path)
        print("successful.")
    except Exception as e:
        print("Error",e)
    return df

In [49]:
salesData=read_csv("./AmazonSalesData/amazon.csv")
print(salesData.head(2))

successful.
   product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 3A Fast Charging 1.5...   

                                            category discounted_price  \
0  Computers&Accessories|Accessories&Peripherals|...             ₹399   
1  Computers&Accessories|Accessories&Peripherals|...             ₹199   

  actual_price discount_percentage rating rating_count  \
0       ₹1,099                 64%    4.2       24,269   
1         ₹349                 43%    4.0       43,994   

                                       about_product  \
0  High Compatibility : Compatible With iPhone 12...   
1  Compatible with all Type C enabled devices, be...   

                                             user_id  \
0  AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...   
1  AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...   

                                           user_na

In [50]:
#1) Check for missing values of individual columns and all columns
def missin_values(df):
    missing=df.isnull().sum() # check for missing values individual columns
    print("missing values in individual columns:\n",missing)
    missing_all=missing.sum() # check for missing values in all columns
    print("missing values in all columns:\n",missing_all)
    return (missing,missing_all)

missing,missing_all=missin_values(salesData)

missing values in individual columns:
 product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64
missing values in all columns:
 2


In [56]:
#  replace missing values with median
# Loop through columns with missing values
for col in salesData.columns[salesData.isnull().any()]:
    if pd.api.types.is_numeric_dtype(salesData[col]):
        median_value = salesData[col].median()
        salesData[col].fillna(median_value, inplace=True)
    else:
        salesData[col].fillna('', inplace=True)

print(salesData.isnull().sum()) # check for missing values after replacing with median
    

product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           0
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64


In [57]:
# 2. Remove duplicate rows using .drop_duplicates() or Excel’s “Remove Duplicates”.
def remove_duplicates(df):
    before=df.shape[0] # check the number of rows before removing duplicates
    print("Before removing duplicates:",before)
    df.drop_duplicates(inplace=True) # remove duplicate rows and replacing
    after=df.shape[0] # check the number of rows after removing duplicates
    print("After removing duplicates:",after)
    duplicates=before-after # check the number of duplicates removed
    if duplicates>0:
        print("Duplicates removed:",duplicates)
    else:
        print("No duplicates found.")
    return df

salesData=remove_duplicates(salesData)

Before removing duplicates: 1465
After removing duplicates: 1465
No duplicates found.


In [59]:
# 3. Standardize text values like gender, country names, etc.
# In the data set it was found that the discounted_price,actual_price, column,discount_percentage,rating_count has some values in string format and some in float format.

#removing ₹  % and ',' symbol and converting the column to float
def standardize_text(df,column):
    df[column] = df[column].replace({'₹': ''}, regex=True) # remove ₹ symbol
    df[column] = df[column].replace({',': ''}, regex=True) # remove ₹ symbol
    df[column]= df[column].replace({'None': '0'}, regex=True) # replace None with 0
    df[column] = df[column].replace({'%': '0'}, regex=True) # replace nan with 0
    df[column] = df[column].astype(float) # convert to float
    return df

salesData=standardize_text(salesData,"discounted_price") # standardizing the discounted_price column
salesData=standardize_text(salesData,"actual_price") # standardizing the actual_price column
salesData=standardize_text(salesData,"discount_percentage") # standardizing the discount_percentage column

salesData.head(2) # check the first two rows of the data set

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,640.0,4.2,24269,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,430.0,4.0,43994,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...


In [60]:
# Summary of the dataset
def print_summary(df):
    print("Data Summary:")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("Data Types:\n", df.dtypes)
    print("First 5 rows:\n", df.head())
    print("Last 5 rows:\n", df.tail())
print_summary(salesData)

Data Summary:
Shape: (1465, 16)
Columns: ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'rating_count', 'about_product', 'user_id', 'user_name', 'review_id', 'review_title', 'review_content', 'img_link', 'product_link']
Data Types:
 product_id              object
product_name            object
category                object
discounted_price       float64
actual_price           float64
discount_percentage    float64
rating                  object
rating_count            object
about_product           object
user_id                 object
user_name               object
review_id               object
review_title            object
review_content          object
img_link                object
product_link            object
dtype: object
First 5 rows:
    product_id                                       product_name  \
0  B07JW9H4J1  Wayona Nylon Braided USB to Lightning Fast Cha...   
1  B098NS6PVG  Ambrane Unbreakable 60W / 

In [61]:
#4.Convert date formats to a consistent type (e.g., dd-mm-yyyy).
def convert_date_format(df,column):
    df[column] = pd.to_datetime(df[column], format='%d-%m-%Y', errors='coerce') # convert to datetime format
    return df

if 'date' in salesData.columns:
    salesData=convert_date_format(salesData,"date") # convert date column to datetime format
else:
    print("date column not found in the dataset so adding the date column")

salesData["date"]=pd.to_datetime("today").date() # adding the date column with today's date
salesData.head(2) # check the first two rows of the data set 

date column not found in the dataset so adding the date column


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link,date
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,640.0,4.2,24269,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,2025-04-21
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,430.0,4.0,43994,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...,2025-04-21


In [62]:
# 5. Rename column headers to be clean and uniform (e.g., lowercase, no spaces).
def standardize_column_names(df):   
    df.columns=df.columns.str.upper() # convert column names to uppercase
    df.columns=df.columns.str.replace(" ","") # remove spaces in column names whicch was observed in the product_id
    return df
salesData=standardize_column_names(salesData)
salesData.info() # check the column names after standardization

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PRODUCT_ID           1465 non-null   object 
 1   PRODUCT_NAME         1465 non-null   object 
 2   CATEGORY             1465 non-null   object 
 3   DISCOUNTED_PRICE     1465 non-null   float64
 4   ACTUAL_PRICE         1465 non-null   float64
 5   DISCOUNT_PERCENTAGE  1465 non-null   float64
 6   RATING               1465 non-null   object 
 7   RATING_COUNT         1465 non-null   object 
 8   ABOUT_PRODUCT        1465 non-null   object 
 9   USER_ID              1465 non-null   object 
 10  USER_NAME            1465 non-null   object 
 11  REVIEW_ID            1465 non-null   object 
 12  REVIEW_TITLE         1465 non-null   object 
 13  REVIEW_CONTENT       1465 non-null   object 
 14  IMG_LINK             1465 non-null   object 
 15  PRODUCT_LINK         1465 non-null   o

In [64]:
#converting the data types of the columns to the specified data types
listOfColumnsWithFloatsPossible=["DISCOUNTED_PRICE","ACTUAL_PRICE","DISCOUNT_PERCENTAGE"] # list of columns with spaces
def convert_data_types(df,column,data_type):
    df[column] = df[column].astype(data_type) # convert column to specified data type
    return df

for column in listOfColumnsWithFloatsPossible:
    salesData=convert_data_types(salesData,column,"float") # convert column to float type

salesData.info() # check the column names after standardization    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PRODUCT_ID           1465 non-null   object 
 1   PRODUCT_NAME         1465 non-null   object 
 2   CATEGORY             1465 non-null   object 
 3   DISCOUNTED_PRICE     1465 non-null   float64
 4   ACTUAL_PRICE         1465 non-null   float64
 5   DISCOUNT_PERCENTAGE  1465 non-null   float64
 6   RATING               1465 non-null   object 
 7   RATING_COUNT         1465 non-null   object 
 8   ABOUT_PRODUCT        1465 non-null   object 
 9   USER_ID              1465 non-null   object 
 10  USER_NAME            1465 non-null   object 
 11  REVIEW_ID            1465 non-null   object 
 12  REVIEW_TITLE         1465 non-null   object 
 13  REVIEW_CONTENT       1465 non-null   object 
 14  IMG_LINK             1465 non-null   object 
 15  PRODUCT_LINK         1465 non-null   o

In [65]:
salesData.head(2) # check the first two rows of the data set

,PRODUCT_ID,PRODUCT_NAME,CATEGORY,DISCOUNTED_PRICE,ACTUAL_PRICE,DISCOUNT_PERCENTAGE,RATING,RATING_COUNT,ABOUT_PRODUCT,USER_ID,USER_NAME,REVIEW_ID,REVIEW_TITLE,REVIEW_CONTENT,IMG_LINK,PRODUCT_LINK,DATE
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,640.0,4.2,24269,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,2025-04-21
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,430.0,4.0,43994,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...,2025-04-21


In [66]:
# saving the cleaned data to a new csv file
salesData.to_csv("./AmazonSalesData/cleaned_amazon.csv",index=False) # save the cleaned data to a new csv file

In [67]:
#reading the cleaned data to check if it is saved correctly
cleanedData=read_csv("./AmazonSalesData/cleaned_amazon.csv")
print(cleanedData.info())
cleanedData.head(2) # check the first two rows of the data set

successful.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PRODUCT_ID           1465 non-null   object 
 1   PRODUCT_NAME         1465 non-null   object 
 2   CATEGORY             1465 non-null   object 
 3   DISCOUNTED_PRICE     1465 non-null   float64
 4   ACTUAL_PRICE         1465 non-null   float64
 5   DISCOUNT_PERCENTAGE  1465 non-null   float64
 6   RATING               1465 non-null   object 
 7   RATING_COUNT         1463 non-null   float64
 8   ABOUT_PRODUCT        1465 non-null   object 
 9   USER_ID              1465 non-null   object 
 10  USER_NAME            1465 non-null   object 
 11  REVIEW_ID            1465 non-null   object 
 12  REVIEW_TITLE         1465 non-null   object 
 13  REVIEW_CONTENT       1465 non-null   object 
 14  IMG_LINK             1465 non-null   object 
 15  PRODUCT_LINK         1465 

,PRODUCT_ID,PRODUCT_NAME,CATEGORY,DISCOUNTED_PRICE,ACTUAL_PRICE,DISCOUNT_PERCENTAGE,RATING,RATING_COUNT,ABOUT_PRODUCT,USER_ID,USER_NAME,REVIEW_ID,REVIEW_TITLE,REVIEW_CONTENT,IMG_LINK,PRODUCT_LINK,DATE
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,399.0,1099.0,640.0,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...,2025-04-21
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,199.0,349.0,430.0,4.0,43994.0,"Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...,2025-04-21
